In [6]:
import re

In [84]:
def QASMDict(gates):
#     print(gates)
    qasmDict = {} #  {'qbit number' : []}
#     print(gates)
    count = []
    for x in range(len(gates)):
        if x == 1 or x == 0:
            continue
        if gates[x][0] == 'qreg':
            for y in range(int(gates[x][1][2])):
                qasmDict[str(y)] = []
#         print(list(qasmDict.keys()))
        if len(count) == 0:
            for b in list(qasmDict.keys()):
                count.append(0)
        print(count)
        if gates[x][0] != 'OPENQASM' and gates[x][0] != 'include' and gates[x][0] != 'qreg' and gates[x][0] != 'creg' and gates[x][0] != 'measure':
            if len(gates[x])-1 > 1:
                if count[int(gates[x][1][2])] > count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][2][2])].append([])
                    count[int(gates[x][2][2])] += 1
                elif count[int(gates[x][1][2])] < count[int(gates[x][2][2])]:
                    qasmDict[str(gates[x][1][2])].append([])
                    count[int(gates[x][1][2])] += 1
                qasmDict[str(gates[x][2][2])].append(gates[x])
                qasmDict[str(gates[x][1][2])].append(gates[x])
            else:
                qasmDict[str(gates[x][1][2])].append(gates[x])
                count[int(gates[x][1][2])] += 1
#             for z in range(len(gates[x])-1):
#                 qasmDict[str(gates[x][z+1][2])].append(gates[x])
#                 count[int(gates[x][z+1][2])] += 1
#         print(count.index(min(count)))
#         print(count.index(max(count)))
        uneven = True
        while uneven:
            uneven = False
            minInd = count.index(min(count))
            maxInd = count.index(max(count))
            if count[maxInd] > count[minInd] + 1:
                qasmDict[str(minInd)].append([])
                count[minInd] += 1
                uneven = True
#         for a in count:
            
    return qasmDict

def openQASM(qasm_loc):
    qasm_f = open(qasm_loc, "r")
    gates = []
    for line in qasm_f:
        gates.append( list(filter(None, re.split(" |,", line.strip()[:-1]))) )
    print(gates)
    qasmDict = QASMDict(gates)
    qasm_f.close()
    return qasmDict


In [92]:
#MAIN SWAPS
def HHtoNone(qubit, depth, circuitDict):
    index2 = depth + 1
    if circuitDict[str(qubit)][depth + 1] == []:
        for x in range(depth+1,len(circuitDict[str(qubit)])):
            if circuitDict[str(qubit)][x] != []:
                index2 = x
                break
    circuitDict[str(qubit)][index2] = [])
    circuitDict[str(qubit)].replace(depth, [])
    return circuitDict


def NonetoHH(qubit, depth, circuitDict):
    qString = 'q[' + str(qubit) + ']'
    circuitDict[str(qubit)].insert(depth, ['h', qString])
    circuitDict[str(qubit)].insert(depth, ['h', qString])
    return circuitDict

def HZHtoCX(qubit, depth, circuitDict):
    return 0

In [93]:
circuit = openQASM('circuit.qasm')
print(circuit)
print(HHtoNone(0,0,circuit))

[['OPENQASM', '2.0'], ['include', '"qelib1.inc"'], ['qreg', 'q[2]'], ['creg', 'c[2]'], ['h', 'q[0]'], ['h', 'q[0]'], ['cz', 'q[1]', 'q[0]'], ['measure', 'q[0]', '->', 'c[0]'], ['measure', 'q[1]', '->', 'c[1]']]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[2, 1]
[2, 2]
[2, 2]
{'0': [['h', 'q[0]'], ['h', 'q[0]'], ['cz', 'q[1]', 'q[0]']], '1': [[], [], ['cz', 'q[1]', 'q[0]']]}


AttributeError: 'list' object has no attribute 'replace'